###Transform Refunds Data
1. Extract specific portion of the string from refund_reason using split function
2. Extract specific portion of the string from refund_reason using regexp_extract function
3. Extract date and time from the refund_timestamp
4. Write transformed data to the silver schema in hive meta store

In [0]:
SELECT refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       refund_reason
FROM hive_metastore.bronze.refunds;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00Z,85.75,Payment Error:Retailer
2,69,2025-01-03T12:40:15Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40Z,150.00,Product Returned:Customer
8,85,2025-01-14T13:15:30Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15Z,250.75,Product Returned:Customer


#####1. Extract specific portion of the string from refund_reason using split function

In [0]:
select refund_id,
       payment_id,
       refund_timestamp,
       refund_amount,
       split(refund_reason, ':')[0] as refund_reason,
       split(refund_reason, ':')[1] as refund_source
from hive_metastore.bronze.refunds;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason,refund_source
1,66,2025-01-10T11:30:00Z,85.75,Payment Error,Retailer
2,69,2025-01-03T12:40:15Z,120.50,Order Cancelled,Customer
3,72,2025-01-06T14:45:30Z,65.00,Product Returned,Customer
4,73,2025-01-07T16:10:45Z,210.99,Order Cancelled,Customer
5,75,2025-01-09T18:25:00Z,45.20,Payment Error,Retailer
6,80,2025-01-10T09:35:20Z,130.15,Order Cancelled,Customer
7,83,2025-01-12T11:20:40Z,150.00,Product Returned,Customer
8,85,2025-01-14T13:15:30Z,89.99,Order Cancelled,Customer
9,89,2025-01-15T15:00:00Z,78.50,Payment Error,Retailer
10,91,2025-01-17T16:45:15Z,250.75,Product Returned,Customer


#####3. Extract date and time from the refund_timestamp

In [0]:
select refund_id,
       payment_id,
       CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) as refund_date,
       date_format(refund_timestamp, 'HH:mm:ss') as refund_time,
       refund_amount,
       split(refund_reason, ':')[0] as refund_reason,
       split(refund_reason, ':')[1] as refund_source
from hive_metastore.bronze.refunds;

refund_id,payment_id,refund_date,refund_time,refund_amount,refund_reason,refund_source
1,66,2025-01-10,11:30:00,85.75,Payment Error,Retailer
2,69,2025-01-03,12:40:15,120.50,Order Cancelled,Customer
3,72,2025-01-06,14:45:30,65.00,Product Returned,Customer
4,73,2025-01-07,16:10:45,210.99,Order Cancelled,Customer
5,75,2025-01-09,18:25:00,45.20,Payment Error,Retailer
6,80,2025-01-10,09:35:20,130.15,Order Cancelled,Customer
7,83,2025-01-12,11:20:40,150.00,Product Returned,Customer
8,85,2025-01-14,13:15:30,89.99,Order Cancelled,Customer
9,89,2025-01-15,15:00:00,78.50,Payment Error,Retailer
10,91,2025-01-17,16:45:15,250.75,Product Returned,Customer


#####4. Write transformed data to the silver schema in hive meta store

In [0]:
-- We have not created the silver schema in the hive_metastore yet. Lets create one

create schema hive_metastore.silver

In [0]:
CREATE TABLE hive_metastore.silver.refunds
AS
select refund_id,
       payment_id,
       CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) as refund_date,
       date_format(refund_timestamp, 'HH:mm:ss') as refund_time,
       refund_amount,
       split(refund_reason, ':')[0] as refund_reason,
       split(refund_reason, ':')[1] as refund_source
from hive_metastore.bronze.refunds;

num_affected_rows,num_inserted_rows


In [0]:
desc extended hive_metastore.silver.refunds

col_name,data_type,comment
refund_id,int,null
payment_id,int,null
refund_date,date,null
refund_time,string,null
refund_amount,"decimal(10,2)",null
refund_reason,string,null
refund_source,string,null
,,
# Delta Statistics Columns,,
Column Names,"refund_id, refund_source, refund_amount, refund_reason, payment_id, refund_time, refund_date",


In [0]:
-- Since we did not specified the location for this managed table that got created, to be written, it stored the data in the default location:
-- dbfs:/user/hive/warehouse/silver.db/refunds

-- Important NOTE: WHen you don't specify the location for a schema, when you create a managed table in that schema, the data will be returned to the default storage, which is the user Hive warehouse directory - that's created as part of the workspace itself.

--e.i, The files will be stored in the user Hive warehouse folder in the root storage and when you delete a workspace, the storage also gets deleted

-- Hence it is always important to create an external location like we did with the unity catalog, and write all your data to external storage -  You can do the same to Hive metastore as well.